In [5]:
from rich import print as rich_print

from langchain.docstore.document import Document
from langchain_community.chat_models import ChatOllama
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain.chains import create_extraction_chain
from typing import Optional, List
from langchain.chains import create_extraction_chain_pydantic
from pydantic import BaseModel
from langchain import hub



In [2]:
local_llm = ChatOllama(model ="mistral")


In [3]:
# RAG
def rag(chunks, collection_name):
    vectorstore = Chroma.from_documents(
        documents=documents,
        collection_name=collection_name,
        embedding=OllamaEmbeddings(model='nomic-embed-text'),
    )
    retriever = vectorstore.as_retriever()

    prompt_template = """Answer the question based only on the following context:
    {context}
    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(prompt_template)

    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | local_llm
        | StrOutputParser()
    )
    result = chain.invoke("What is the use of Text Splitting?")
    rich_print(result)


In [4]:
# 1. Character Text Splitting
print("#### Character Text Splitting ####")

text = "Text splitting in LangChain is a critical feature that facilitates the division of large texts into smaller, manageable segments. "

# Manual Splitting
chunks = []
chunk_size = 35 # Characters
for i in range(0, len(text), chunk_size):
    chunk = text[i:i + chunk_size]
    chunks.append(chunk)
documents = [Document(page_content=chunk, metadata={"source": "local"}) for chunk in chunks]
rich_print(documents)

# Automatic Text Splitting
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size = 35, chunk_overlap=0, separator='', strip_whitespace=False)
documents = text_splitter.create_documents([text])
rich_print(documents)

#### Character Text Splitting ####


[
    Document(metadata={'source': 'local'}, page_content='Text splitting in LangChain is a cr'),
    Document(metadata={'source': 'local'}, page_content='itical feature that facilitates the'),
    Document(metadata={'source': 'local'}, page_content=' division of large texts into small'),
    Document(metadata={'source': 'local'}, page_content='er, manageable segments. ')
]

[
    Document(metadata={}, page_content='Text splitting in LangChain is a cr'),
    Document(metadata={}, page_content='itical feature that facilitates the'),
    Document(metadata={}, page_content=' division of large texts into small'),
    Document(metadata={}, page_content='er, manageable segments. ')
]

In [5]:
# 2. Recursive Character Text Splitting
print("#### Recursive Character Text Splitting ####")

from langchain.text_splitter import RecursiveCharacterTextSplitter
with open('E:\git\AI-Algorithm-Journey\chunking_text.txt', 'r', encoding='utf-8') as file:
    text = file.read()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 65, chunk_overlap=0) # ["\n\n", "\n", " ", ""] 65,450
rich_print(text_splitter.create_documents([text])) 

#### Recursive Character Text Splitting ####


[
    Document(metadata={}, page_content='Text splitting in LangChain is a critical feature that'),
    Document(metadata={}, page_content='facilitates the division of large texts into smaller, manageable'),
    Document(metadata={}, page_content='segments.'),
    Document(metadata={}, page_content='This capability is vital for improving comprehension and'),
    Document(metadata={}, page_content='processing efficiency, especially in tasks that require detailed'),
    Document(metadata={}, page_content='analysis or extraction of specific contexts.'),
    Document(metadata={}, page_content='ChatGPT, developed by OpenAI, represents a leap forward in'),
    Document(metadata={}, page_content='natural language processing technologies.'),
    Document(metadata={}, page_content="It's a conversational AI model capable of understanding and"),
    Document(metadata={}, page_content='generating human-like text, allowing for dynamic interactions'),
    Document(metadata={}, page_content='and providing responses that are remarkably coherent and'),
    Document(metadata={}, page_content='contextually relevant. ChatGPT has been integrated into a'),
    Document(metadata={}, page_content='multitude of applications, revolutionizing the way we interact'),
    Document(metadata={}, page_content='with machines and access information.'),
    Document(metadata={}, page_content='By leveraging LangChain for text splitting, users can'),
    Document(metadata={}, page_content='efficiently navigate and analyze vast amounts of text data,'),
    Document(metadata={}, page_content='facilitating a deeper understanding and more insightful'),
    Document(metadata={}, page_content='conclusions.')
]

In [6]:
# 3. Document Specific Splitting
print("#### Document Specific Splitting ####")

# Document Specific Splitting - Markdown
from langchain.text_splitter import MarkdownTextSplitter
splitter = MarkdownTextSplitter(chunk_size = 40, chunk_overlap=0)
markdown_text = """
# Fun in California

## Driving

Try driving on the 1 down to San Diego

### Food

Make sure to eat a burrito while you're there

## Hiking

Go to Yosemite
"""
rich_print(splitter.create_documents([markdown_text]))

#### Document Specific Splitting ####


[
    Document(metadata={}, page_content='# Fun in California\n\n## Driving'),
    Document(metadata={}, page_content='Try driving on the 1 down to San Diego'),
    Document(metadata={}, page_content='### Food'),
    Document(metadata={}, page_content="Make sure to eat a burrito while you're"),
    Document(metadata={}, page_content='there'),
    Document(metadata={}, page_content='## Hiking\n\nGo to Yosemite')
]

In [7]:
# Document Specific Splitting - Python
from langchain.text_splitter import PythonCodeTextSplitter
python_text = """
class Person:
  def __init__(self, name, age):
    self.name = name
    self.age = age

p1 = Person("John", 36)

for i in range(10):
    print (i)
"""
python_splitter = PythonCodeTextSplitter(chunk_size=100, chunk_overlap=0)
rich_print(python_splitter.create_documents([python_text]))


[
    Document(
        metadata={},
        page_content='class Person:\n  def __init__(self, name, age):\n    self.name = name\n    self.age = age'
    ),
    Document(metadata={}, page_content='p1 = Person("John", 36)\n\nfor i in range(10):\n    print (i)')
]

In [8]:
# Document Specific Splitting - Javascript
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language
javascript_text = """
// Function is called, the return value will end up in x
let x = myFunction(4, 3);

function myFunction(a, b) {
// Function returns the product of a and b
  return a * b;
}
"""
js_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.JS, chunk_size=65, chunk_overlap=0
)
rich_print(js_splitter.create_documents([javascript_text]))

[
    Document(metadata={}, page_content='// Function is called, the return value will end up in x'),
    Document(metadata={}, page_content='let x = myFunction(4, 3);'),
    Document(metadata={}, page_content='function myFunction(a, b) {'),
    Document(metadata={}, page_content='// Function returns the product of a and b\n  return a * b;\n}')
]

In [1]:
import os
openapi_key = os.environ.get('OPENAI_API_KEY')


In [10]:
# 4. Semantic Chunking
print("#### Semantic Chunking ####")

from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
with open('E:\git\AI-Algorithm-Journey\chunking_text.txt', 'r', encoding='utf-8') as file:
    text = file.read() 

# Percentile - all differences between sentences are calculated, and then any difference greater than the X percentile is split
text_splitter = SemanticChunker(OpenAIEmbeddings(api_key=openapi_key))
text_splitter = SemanticChunker(
    OpenAIEmbeddings(), breakpoint_threshold_type="percentile" # "standard_deviation", "interquartile"
)
documents = text_splitter.create_documents([text])
rich_print(documents)

#### Semantic Chunking ####


[
    Document(
        metadata={},
        page_content='Text splitting in LangChain is a critical feature that facilitates the division of large 
texts into smaller, manageable segments. This capability is vital for improving comprehension and processing 
efficiency, especially in tasks that require detailed analysis or extraction of specific contexts.'
    ),
    Document(
        metadata={},
        page_content="ChatGPT, developed by OpenAI, represents a leap forward in natural language processing 
technologies. It's a conversational AI model capable of understanding and generating human-like text, allowing for 
dynamic interactions and providing responses that are remarkably coherent and contextually relevant. ChatGPT has 
been integrated into a multitude of applications, revolutionizing the way we interact with machines and access 
information. By leveraging LangChain for text splitting, users can efficiently navigate and analyze vast amounts of
text data, facilitating a deeper understanding and more insightful conclusions."
    )
]

In [6]:
# 5. Agentic Chunking
print("#### Proposition-Based Chunking ####")


class Sentences(BaseModel):
    sentences: List[str]

text = "Text splitting in LangChain is a critical feature that facilitates the division of large texts into smaller, manageable segments. "
with open('E:\git\AI-Algorithm-Journey\chunking_text.txt', 'r', encoding='utf-8') as file:
    text = file.read() 


# https://arxiv.org/pdf/2312.06648.pdf
obj = hub.pull("wfh/proposal-indexing")
llm = ChatOpenAI(model='gpt-3.5-turbo').with_structured_output(Sentences)
runnable = obj | llm



# Extraction
# extraction_chain = create_extraction_chain_pydantic(pydantic_schema=Sentences, llm=llm)
def get_propositions(text):
    # Create an LLM with structured output based on the Sentences schema
    # structured_llm = runnable.with_structured_output(Sentences)

    # Invoke the LLM with the input text and get the structured output
    runnable_output = runnable.invoke({
        "input": text
    })

    # Extract sentences from the structured output
    propositions = runnable_output.sentences

    return propositions
paragraphs = text.split("\n")
text_propositions = []
for i, para in enumerate(paragraphs[:5]):
    propositions = get_propositions(para)
    text_propositions.extend(propositions)
    print (f"Done with {i}")


rich_print (f"You have {len(text_propositions)} propositions")
rich_print(text_propositions[:10])


#### Proposition-Based Chunking ####


c:\Users\johns\anaconda3\envs\chunking\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Done with 0
Done with 1
Done with 2
Done with 3


In [13]:
print("#### Agentic Chunking ####")

from agentic_chunker import AgenticChunker
ac = AgenticChunker()
ac.add_propositions(text_propositions)
rich_print(ac.pretty_print_chunks())
chunks = ac.get_chunks(get_type='list_of_strings')
rich_print(chunks)
documents = [Document(page_content=chunk, metadata={"source": "local"}) for chunk in chunks]
rag(documents, "agentic-chunks")

#### Agentic Chunking ####


C:\Users\johns\AppData\Local\Temp\ipykernel_22360\3972065590.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from agentic_chunker import AgenticChunker


Adding: 'Text splitting in LangChain is a critical feature.'

No chunks, creating a new one

Created new chunk (112bf): Text Splitting

...

Propositions:

-It has revolutionized the way we interact with machines and access information.

None

[
    'Text splitting in LangChain is a critical feature.',
    'Text splitting facilitates the division of large texts into smaller, manageable segments.',
    'This capability is vital for improving comprehension and processing efficiency.',
    'This is especially important in tasks that require detailed analysis or extraction of specific contexts.',
    'The sky is blue and the sun is shining brightly.',
    'ChatGPT represents a leap forward in natural language processing technologies.',
    'ChatGPT was developed by OpenAI.',
    "It's a conversational AI model capable of understanding and generating human-like text.",
    'It allows for dynamic interactions and provides responses that are remarkably coherent and contextually 
relevant.',
    'ChatGPT has been integrated into a multitude of applications.',
    'It has revolutionized the way we interact with machines and access information.'
]

The use of Text Splitting, as mentioned in the provided context, facilitates the division of large texts into 
smaller, manageable segments. This capability is crucial for improving comprehension and processing efficiency, 
particularly within a conversational AI model like LangChain.